In [33]:
# !pip install nbimporter


In [34]:
import os
import random
import numpy as np
from collections import deque
import copy
import cv2
import nbimporter
import SRGAN_body 
import Model_A_Binary_Classifier 

from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.nn.utils import clip_grad_norm_, spectral_norm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split, DataLoader, Dataset
from tqdm import tqdm


In [35]:
from SRGAN_body import weights_init, calculate_psnr, \
    Generator, Discriminator, FeatureExtractor

In [36]:
from Model_A_Binary_Classifier import set_seed, AnimalImage, get_transformation, get_test_transformation

In [37]:
class MergedDataset(Dataset):
    def __init__(self, dataset_32, dataset_128):
        self.dataset_32 = dataset_32
        self.dataset_128 = dataset_128
        assert len(dataset_32) == len(dataset_128), "Datasets must be the same size"

    def __len__(self):
        return len(self.dataset_32)

    def __getitem__(self, idx):
        data_32 = self.dataset_32[idx]
        data_128 = self.dataset_128[idx]
        return (data_128, data_32)

In [38]:
def save_images(hr_image, lr_image, fake_image, label, epoch, show=False):
    lr_image = np.transpose(lr_image.cpu().numpy(), (1, 2, 0))
    hr_image = np.transpose(hr_image.cpu().numpy(), (1, 2, 0))
    fake_image = np.transpose(fake_image.cpu().numpy(), (1, 2, 0))

    name = "Cat" if label == 1 else "Dog"
    cv2.imwrite(f"results/{name}_{epoch}_generated.png", fake_image * 255)
    cv2.imwrite(f"results/{name}_{epoch}_hr.png", hr_image * 255)
    cv2.imwrite(f"results/{name}_{epoch}_lr.png", lr_image * 255)

    if show:
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs = axs.ravel()

        axs[0].imshow(lr_image, cmap='gray')
        axs[0].set_title('Low Resolution')

        axs[1].imshow(hr_image, cmap='gray')
        axs[1].set_title('High Resolution')

        axs[2].imshow(fake_image, cmap='gray')
        axs[2].set_title('Generated Image')

        for ax in axs:
            ax.set_xticks([])
            ax.set_yticks([])

        # fig.suptitle(definitions[label])
        plt.show()


In [39]:
def apply_spectral_norm(model):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.utils.spectral_norm(module)

In [40]:
def train_SRGAN(gen, disc, feature_extractor, train_loader, val_loader,
                epochs, device="cpu"):
    gen.train()
    disc.train()
    best_psnr = 0
    epochs_since_improvement = 0

    # Define loss functions
    criterion_GAN = torch.nn.BCEWithLogitsLoss().to(device)
    criterion_content = torch.nn.MSELoss().to(device)
    criterion_feature_matching = torch.nn.MSELoss().to(device)

    # Setup AdamW optimizers with different learning rates
    optimizer_G = torch.optim.AdamW(gen.parameters(), lr=1e-5, betas=(0.5, 0.999))
    optimizer_D = torch.optim.AdamW(disc.parameters(), lr=1e-6, betas=(0.5, 0.999))

    # Scheduler for the optimizers (Reduce learning rate when a metric has stopped improving)
    scheduler_G = ReduceLROnPlateau(optimizer_G, mode='min', factor=0.1, patience=10, verbose=True)
    scheduler_D = ReduceLROnPlateau(optimizer_D, mode='min', factor=0.1, patience=10, verbose=True)

    # Initialize historical discriminators
    historical_discriminators = []
    historical_disc = Discriminator().to(device)  # Create one instance to be used for all historical states

    # Training loop
    for epoch in range(epochs):
        genLossSum = 0
        disLossSum = 0

        for i, (high_data, low_data) in enumerate(tqdm(train_loader)):
            # Transfer data to device
            low_res = low_data["img"].to(device)
            high_res = high_data["img"].to(device)

            # Adversarial ground truths with label smoothing and noise
            valid = torch.FloatTensor(high_res.size(0), 1).uniform_(0.7, 1.2).to(device)
            fake = torch.FloatTensor(high_res.size(0), 1).uniform_(0.0, 0.3).to(device)
            valid = torch.clamp(valid, 0, 1)
            fake = torch.clamp(fake, 0, 1)

            # ------------------
            #  Train Generator
            # ------------------
            optimizer_G.zero_grad()

            # Generate a high resolution image from low resolution input
            gen_high_res = gen(low_res)

            # Adversarial loss
            validity = disc(gen_high_res)
            loss_GAN = criterion_GAN(validity, valid)

            # Content loss
            loss_content = criterion_content(gen_high_res, high_res)

            # Feature matching loss
            real_features = feature_extractor(high_res).detach()
            fake_features = feature_extractor(gen_high_res)
            loss_feature_matching = criterion_feature_matching(fake_features, real_features)

            # Total loss for the generator
            loss_G = loss_content + 0.001 * loss_GAN + 0.006 * loss_feature_matching
            loss_G.backward()
            clip_grad_norm_(gen.parameters(), 1)  # Clip gradients for generator
            optimizer_G.step()

            # ---------------------
            #  Train Discriminator
            # ---------------------
            optimizer_D.zero_grad()

            # Measure discriminator's ability to classify real from generated samples
            real_loss = criterion_GAN(disc(high_res), valid)
            fake_loss = criterion_GAN(disc(gen_high_res.detach()), fake)
            loss_D = (real_loss + fake_loss) / 2

            # Historical averaging
            historical_losses = []
            for disc_state in historical_discriminators:
                historical_disc.load_state_dict(disc_state)
                historical_disc.eval()  # Set to evaluation mode for inference
                with torch.no_grad():  # No need to track gradients for historical discriminators
                    historical_losses.append(criterion_GAN(historical_disc(high_res), valid))

            if historical_losses:
                historical_loss = sum(historical_losses) / len(historical_losses)
                loss_D += 0.001 * historical_loss

            loss_D.backward()
            clip_grad_norm_(disc.parameters(), 1)  # Clip gradients for discriminator
            optimizer_D.step()

            # Save the current discriminator state
            if len(historical_discriminators) >= 10:
                historical_discriminators.pop(0)
            historical_discriminators.append(copy.deepcopy(disc.state_dict()))

            # Calculate the losses
            genLossSum += loss_G.item()
            disLossSum += loss_D.item()

        # Update learning rates
        scheduler_G.step(genLossSum)
        scheduler_D.step(disLossSum)
        
         # Clear GPU cache to avoid memory build-up
        torch.cuda.empty_cache()

        # Print the losses
        print(
            f"Epoch {epoch + 1}/{epochs} - Generator Loss: {genLossSum / (i + 1)} - Discriminator Loss: {disLossSum / (i + 1)}")

        psnr_values = []
        gen.eval()  # Set the generator to evaluation mode
        rnd = random.randint(0, batch_size - 1)
        with torch.no_grad():
            for i, (high_res, low_res) in enumerate(val_loader):
                high_res_real = high_res["img"].to(device)
                label = low_res["label"]
                low_res = low_res["img"].to(device)
                high_res_fake = gen(low_res)
                # Save some sample images
                if i == 0:  # Change this condition to save images as needed
                    save_images(high_res_real[rnd], low_res[rnd], high_res_fake[rnd], label[rnd], epoch)

                # Calculate PSNR
                psnr_value = calculate_psnr(high_res_real, high_res_fake, device)
                psnr_values.append(psnr_value)

        avg_psnr = sum(psnr_values) / len(psnr_values)
        print(f"Average PSNR on validation set for epoch {epoch + 1}: {avg_psnr} dB")

        # Check for improvement
        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            epochs_since_improvement = 0
            torch.save(gen.state_dict(), 'best_generator.pth')
            torch.save(disc.state_dict(), 'best_discriminator.pth')
        else:
            epochs_since_improvement += 1

        # Early stopping
        if epochs_since_improvement == 10:
            print("No improvement in PSNR for 10 consecutive epochs, stopping training.")
            break

        # Set the generator back to train mode
        gen.train()

In [41]:
if __name__ == "__main__":

    batch_size = 32
    seed = 10
    set_seed(seed)
    # Create dataset and split it into train, and validation
    cat_dog_dataset = AnimalImage(data_root="/home/mdnurualabsarsiddiky/Desktop/Absar/ECGR8119/Midterm_2024/dogs_vs_cats/train",
                                   transformation=get_test_transformation(size=128))
    train_data_128, val_data_128, _ = random_split(cat_dog_dataset, [0.65, 0.05, 0.3],
                                                   generator=torch.Generator().manual_seed(seed))

    # Create dataset and split it into train, and validation
    cat_dog_dataset = AnimalImage(data_root="/home/mdnurualabsarsiddiky/Desktop/Absar/ECGR8119/Midterm_2024/dogs_vs_cats/train",
                                   transformation=get_test_transformation(size=32))
    train_data_32, val_data_32, _ = random_split(cat_dog_dataset, [0.65, 0.05, 0.3],
                                                 generator=torch.Generator().manual_seed(seed))

    merged_train_data = MergedDataset(train_data_32, train_data_128)
    merged_val_data = MergedDataset(val_data_32, val_data_128)

    train_loader = DataLoader(merged_train_data, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=False,
                              num_workers=4)
    val_loader = DataLoader(merged_val_data, batch_size=batch_size, shuffle=True, pin_memory=False, num_workers=4,
                            drop_last=False)

    # Sets string definitions based on integer values
    definitions = {0: "Dog", 1: "Cat"}
    samples = 10
    epochs = 150
    lr_g = 1e-3  # Learning rate for generator
    lr_d = 1e-4  # Learning rate for discriminator
    device = "cuda:0"
    # Create the generator
    generator = Generator(num_residual_blocks=24).to(device)
    # Run a forward pass with a dummy 32 to initialize parameters
    dummy_input = torch.randn(1, 3, 32, 32).to(device)  # Adjust the size as necessary for your model
    generator(dummy_input)
    # Apply the weight initialization
    generator.apply(weights_init)
    # Create the discriminator
    discriminator = Discriminator().to(device)
    dummy_input = torch.randn(1, 3, 128, 128).to(device)  # The size should match the discriminator's expected input
    discriminator(dummy_input)
    discriminator.apply(weights_init)
    # Instantiate the feature extractor
    feature_extractor = FeatureExtractor().eval()  # Set to evaluation mode

    # Move to the device and make sure to not track gradients
    feature_extractor.to('cuda' if torch.cuda.is_available() else 'cpu')
    for parameter in feature_extractor.parameters():
        parameter.requires_grad = False

    train_SRGAN(gen=generator, disc=discriminator, feature_extractor=feature_extractor,
                train_loader=train_loader, val_loader=val_loader, epochs=epochs,
                device=device)


Seed set to 10
100%|██████████| 507/507 [03:28<00:00,  2.43it/s]

Epoch 1/150 - Generator Loss: 8.445867410072914 - Discriminator Loss: 0.6236695330994134


Average PSNR on validation set for epoch 1: 46.24361801147461 dB


100%|██████████| 507/507 [03:26<00:00,  2.46it/s]

Epoch 2/150 - Generator Loss: 3.8256716229741623 - Discriminator Loss: 0.4947392616634068


...

Epoch 149/150 - Generator Loss: 0.5936466058095297 - Discriminator Loss: 0.36690531591691916


Average PSNR on validation set for epoch 149: 55.738800048828125 dB


100%|██████████| 507/507 [03:25<00:00,  2.47it/s]

Epoch 150/150 - Generator Loss: 0.5916595290280893 - Discriminator Loss: 0.3650570534154978


Average PSNR on validation set for epoch 150: 55.7865104675293 dB


In [31]:
import os
print(os.cpu_count())  # Number of CPU cores

64


In [42]:
import torch

# Print GPU memory usage
gpu_memory = torch.cuda.memory_allocated() / (1024 ** 2)
print(f"GPU Memory Usage: {gpu_memory:.2f} MB")

# Track total GPU memory used, including cached memory
total_gpu_memory = torch.cuda.memory_reserved() / (1024 ** 2)
print(f"Total GPU Memory Reserved: {total_gpu_memory:.2f} MB")


GPU Memory Usage: 596.15 MB
Total GPU Memory Reserved: 1496.00 MB
